Name: Maurya Vijayaramachandran 

Project: implementing the resnet architecture. 

Import the libraries

In [ ]:
import os
import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras import layers 
from tensorflow.keras.datasets import mnist

Loading the data

In [ ]:
(x_train, y_train),(x_test,y_test) = mnist.load_data()

11490434/11490434 [==============================] - 1s 0us/step


Load the dataset and split it into test and train sets

The test and train sets are further split into

a. Features X_train , X_test 

b. Labels y_train, y_test

In [ ]:
x_train = x_train.reshape(-1,28,28,1).astype("float32")/255.0
x_test = x_test.reshape(-1,28,28,1).astype("float32")/255.0

Analyzing the data

In [ ]:
x_train.shape

(60000, 28, 28, 1)

Creating a dense layer 

In [ ]:
class Dense(layers.Layer):
  """
    A dense layer applies a linear transformation to the input data and produces an output of a specified size.

    Attributes:
        units: An integer representing the number of units in the layer.

    Methods:
        __init__(self, units): Initializes the Dense object with the specified number of units.
        build(self, input_shape): Creates and initializes the trainable parameters of the layer.
        call(self, inputs): Defines the forward pass of the layer.

    Example usage:
        dense_layer = Dense(units=64)
  """
  def __init__(self,units):
        """
        Initializes the Dense object with the specified number of units.

        Args:
            units: An integer representing the number of units in the layer.
        """
    super().__init__()
    self.units = units

  def build(self, input_shape):
         """
        Creates and initializes the trainable parameters of the layer.

        Args:
            input_shape: A tuple representing the shape of the input to the layer.

        Returns:
            None
        """
        self.w = self.add_weight(
        name = "w",
        shape = (input_shape[-1], self.units),
        initializer = "random_normal",
        trainable = True,

    )
        self.b = self.add_weight(
        name = "b",
        shape = (self.units,),
        initializer= "zeros",
        trainable = True,
    )
  def call(self, inputs):
        """
        Defines the forward pass of the layer.

        Args:
            inputs: The input tensor to the layer.

        Returns:
            A tensor representing the output of the layer.
        """
    return tf.matmul(inputs, self.w)+ self.b

Creating the CNN block

In [ ]:
class CNNBlock(layers.Layer):
 """
    A convolutional block consisting of a convolutional layer, batch normalization, and a ReLU activation function.

    Attributes:
        out_channels: An integer representing the number of output channels in the convolutional layer.
        kernel_size: An integer or tuple/list of 2 integers representing the height and width of the convolutional kernel.

    Methods:
        __init__(self, out_channels, kernel_size=3): Initializes the CNNBlock object with the specified number of output channels and kernel size.
        call(self, input_tensor, training=False): Defines the forward pass of the layer.

    Example usage:
        cnn_block = CNNBlock(out_channels=64, kernel_size=(3, 3))
"""
  def __init__(self, out_channels, kernel_size = 3):
        """
        Initializes the CNNBlock object with the specified number of output channels and kernel size.

        Args:
            out_channels: An integer representing the number of output channels in the convolutional layer.
            kernel_size: An integer or tuple/list of 2 integers representing the height and width of the convolutional kernel.
                Default is 3.

        Returns:
            None
        """
    super().__init__()
    self.conv = layers.Conv2D(out_channels, kernel_size,padding = "same")
    self.bn = layers.BatchNormalization()
    self.relu = MyReLU()
  def call(self,input_tensor, training = False):
    """
        Defines the forward pass of the layer.

        Args:
            input_tensor: The input tensor to the layer.
            training: A boolean indicating whether the layer is in training mode. Default is False.

        Returns:
            A tensor representing the output of the layer.
    """
    x = self.conv(input_tensor)
    x = self.bn(x, training = training)
    x = self.relu(x)
    return x

Making the Rectified Linear Unit Activation function

In [ ]:
class MyReLU(layers.Layer):
    """
    A custom ReLU activation function.

    Methods:
        __init__(self): Initializes the MyReLU object.
        call(self, x): Defines the forward pass of the layer.

    Example usage:
        my_relu = MyReLU()
    """

    def __init__(self):
        """
        Initializes the MyReLU object.

        Returns:
            None
        """
        super().__init__()

    def call(self, x):
        """
        Defines the forward pass of the layer.

        Args:
            x: The input tensor to the layer.

        Returns:
            A tensor representing the output of the layer.
        """
        return tf.math.maximum(x, 0)


In [ ]:
# model = keras.Sequential(
#     [
#      CNNBlock(32),
#      CNNBlock(64),
#      CNNBlock(128),
#      layers.Flatten(),
#      layers.Dense(10)
#     ]
# )

Write definition here 

In [ ]:
class ResBlock(layers.Layer):
    """Docstring here
  """
  def __init__(self, channels):
    super().__init__()
    self.cnn1 = CNNBlock(channels[0])
    self.cnn2 = CNNBlock(channels[1])
    self.cnn3 = CNNBlock(channels[2])
    self.pooling = layers.MaxPooling2D()
    self.identity_mapping = layers.Conv2D(channels[1],kernel_size =1, padding = "same" )
  def call(self, input_tensor, training = False):
    x = self.cnn1(input_tensor, training = training)
    x = self.cnn2(x, training = training)
    x = self.cnn3(
        x+self.identity_mapping(input_tensor), training = training
    )
    return self.pooling(x)

Creating the residual blocks

In [ ]:
class ResBlock(layers.Layer):
    """
    A residual block consisting of three convolutional blocks, a max pooling layer, and an identity mapping.

    Attributes:
        channels: A tuple of 3 integers representing the number of output channels for each of the convolutional blocks.

    Methods:
        __init__(self, channels): Initializes the ResBlock object with the specified number of output channels for each block.
        call(self, input_tensor, training=False): Defines the forward pass of the layer.

    Example usage:
        res_block = ResBlock(channels=(64, 64, 256))
    """

    def __init__(self, channels):
        """
        Initializes the ResBlock object with the specified number of output channels for each block.

        Args:
            channels: A tuple of 3 integers representing the number of output channels for each of the convolutional blocks.

        Returns:
            None
        """
        super().__init__()
        self.cnn1 = CNNBlock(channels[0])
        self.cnn2 = CNNBlock(channels[1])
        self.cnn3 = CNNBlock(channels[2])
        self.pooling = layers.MaxPooling2D()
        self.identity_mapping = layers.Conv2D(channels[1], kernel_size=1, padding="same")

    def call(self, input_tensor, training=False):
        """
        Defines the forward pass of the layer.

        Args:
            input_tensor: The input tensor to the layer.
            training: A boolean indicating whether the layer is in training mode. Default is False.

        Returns:
            A tensor representing the output of the layer.
        """
        x = self.cnn1(input_tensor, training=training)
        x = self.cnn2(x, training=training)
        x = self.cnn3(x + self.identity_mapping(input_tensor), training=training)
        return self.pooling(x)


Defining the model and passing in the number of classes. 

1. It is 10 for MNIST

In [ ]:
model = reslike(num_classes = 10)

The compilation block

In [ ]:

model.compile(
    optimizer = keras.optimizers.Adam(),
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits= True),
    metrics= ["accuracy"],
)

The training block

In [ ]:

model.fit(x_train,y_train, batch_size = 64, epochs=3, verbose=2)

Epoch 1/3


938/938 - 43s - loss: 0.1060 - accuracy: 0.9706 - 43s/epoch - 46ms/step
Epoch 2/3
938/938 - 30s - loss: 0.0376 - accuracy: 0.9885 - 30s/epoch - 32ms/step
Epoch 3/3
938/938 - 28s - loss: 0.0311 - accuracy: 0.9902 - 28s/epoch - 30ms/step


The evaluation block 

In [ ]:

model.evaluate(x_test, y_test, batch_size = 64, verbose=2)

157/157 - 2s - loss: 0.1043 - accuracy: 0.9725 - 2s/epoch - 14ms/step


[0.10434883087873459, 0.9725000262260437]

Saving the model

In [ ]:
model.save_weights("/content/model/")

In [ ]:
model.model_summary().summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 res_block (ResBlock)        (None, 14, 14, 64)        28640     
                                                                 
 res_block_2 (ResBlock)      (None, 7, 7, 512)         1569408   
                                                                 
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 10)                5130      
                                                                 
Total params: 1,603,178
Trainable params: 1,601,130
Non-trainable params: 2,048
_______________________________________________